https://github.com/monologg/KoELECTRA

In [1]:
# import library
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

In [2]:
# ice_data ='https://raw.githubusercontent.com/lhshs/mydataset/main/genia/2307_NLP/ice_model.csv'
# tsh_data = 'https://raw.githubusercontent.com/lhshs/mydataset/main/genia/2307_NLP/tsh_model.csv'
# ice = pd.read_csv(ice_data, index_col=0)
# tsh = pd.read_csv(ice_data, index_col=0)

In [3]:

ice = pd.read_csv('data/icecream_emot.csv')
tsh = pd.read_csv('data/sherpa_emot3.csv')

tsh = tsh[['title_comments', 'star']]
tsh.rename(columns={'title_comments' : 'text', 'star' : 'score'}, inplace=True)
tsh['score'] = tsh['score'].apply(lambda x : int(x.replace('점','')))
tsh['score'] = tsh['score'].apply(lambda x : 0 if x == 3 else (1 if x > 3 else -1))

ice = ice[['title_comments', 'rated']]
ice.rename(columns={'title_comments' : 'text', 'rated' : 'score'}, inplace=True)
ice['score'] = ice['score'].apply(lambda x : int(x[-1]))
ice['score'] = ice['score'].apply(lambda x : 0 if x == 3 else (1 if x > 3 else -1))


In [5]:
ice

,text,score
0,알차네요 무엇보다도 빨리들을 수 있어 좋습니다,1
1,명불허전 명강사라 그런지 말씀을 참 잘하시네요~,1
2,강추 30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 ...,1
3,패들렛과 띵커벨 활용 잘 할 수 있어요 패들렛과 띵커벨 활용하고 있었으나 다양한 교...,1
4,"필수 연수 부담 없이 편안하게 안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안...",1
...,...,...
5236,알베르토가 함께하는 핵심역량 연계 다문화 교육 여타 연수와 다르게 새로 알게된 내용...,1
5237,전문가가 알려주는 창의 영재교육의 비법 연수 후기 영재 교육에 대한 틀을 잡아주시는...,1
5238,도도한 교사생활 연수를 통해 자신감을 더 갖게 되었다.,1
5239,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요. 교실에서 활용할 수 있는...,1


ice modeling

In [27]:

model_lst = ['jaehyeong/koelectra-base-v3-generalized-sentiment-analysis', 
            'klue/bert-base', 
            'matthewburke/korean_sentiment',
            # 'monologg/koelectra-base-discriminator',
            # "monologg/koelectra-small-discriminator",
            "monologg/koelectra-base-v2-discriminator",
            # "monologg/koelectra-small-v2-discriminator",
            # "monologg/koelectra-base-v3-discriminator",
            # "monologg/koelectra-small-v3-discriminator",
            # "facebook/bart-large-mnli",
            # "monologg/koelectra-base-v3-goemotions",
            'skt/kogpt2-base-v2',
            # 'skt/kobert-base-v1',
            'beomi/kcbert-large',
            # "google/electra-small-discriminator"
            ]


for i in model_lst: # 0,1,2,3
    # load model
    pre_model = i
    tokenizer = AutoTokenizer.from_pretrained(pre_model)
    model = AutoModelForSequenceClassification.from_pretrained(pre_model)
    sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

    print('====='+ '\033[95m' + pre_model + '\033[0m' + '=====')

    # target reviews
    review_list = list(ice.text)

    ice[pre_model] = '-'
    # predict
    for idx, review in tqdm(enumerate(review_list)):
        pred = sentiment_classifier(review)
        # print(f'{review}\n>> {pred[0]}')
        if ((pred[0]['label']=='1')&(pred[0]['score']>0.7)) | ((pred[0]['label']=='LABEL_1')&(pred[0]['score']>0.7)):
            ice[pre_model][idx] = '1'
        elif ((pred[0]['label']=='0')&(pred[0]['score']>0.7)) | ((pred[0]['label']=='LABEL_0')&(pred[0]['score']>0.7)):  
            ice[pre_model][idx] = '-1'
        else:
            ice[pre_model][idx] = '0'

        # print(f'{review}\n>> {pred[0]}')

print('🥕')
print('')


=====jaehyeong/koelectra-base-v3-generalized-sentiment-analysis=====


0it [00:00, ?it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_5544\1809579344.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[pre_model][idx] = '1'
3it [00:00, 21.60it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_5544\1809579344.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[pre_model][idx] = '-1'
12it [00:00, 20.67it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_5544\1809579344.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

=====klue/bert-base=====


5241it [04:24, 19.85it/s]


=====matthewburke/korean_sentiment=====


...

=====skt/kogpt2-base-v2=====


5241it [04:35, 19.01it/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=====beomi/kcbert-large=====


5241it [11:59,  7.28it/s]

🥕



In [29]:
for i in ice.iloc[:, :].columns:
    print(ice[i].value_counts())

유익한 연수였습니다. 유익한 연수였습니다.                                                                                                                                                                        11
추천합니다, 추천합니다,                                                                                                                                                                                   7
많은 도움이 되었습니다 ^^ 많은 도움이 되었습니다 ^^                                                                                                                                                                 6
연수시간,,, 좋아요!! 괜찮네요 연수시간,,, 좋아요!! 괜찮네요                                                                                                                                                           6
추천합니다. 유용한 강좌입니다. 추천합니다.                                                                                                                                                                        5
                              

In [30]:
# ice.to_csv('ice_model.csv')

tsh modeling

In [6]:

model_lst = ['jaehyeong/koelectra-base-v3-generalized-sentiment-analysis', 
            'klue/bert-base', 
            'matthewburke/korean_sentiment',
            # 'monologg/koelectra-base-discriminator',
            # "monologg/koelectra-small-discriminator",
            "monologg/koelectra-base-v2-discriminator",
            # "monologg/koelectra-small-v2-discriminator",
            # "monologg/koelectra-base-v3-discriminator",
            # "monologg/koelectra-small-v3-discriminator",
            # "facebook/bart-large-mnli",
            # "monologg/koelectra-base-v3-goemotions",
            'skt/kogpt2-base-v2',
            # 'skt/kobert-base-v1',
            'beomi/kcbert-large',
            # "google/electra-small-discriminator"
            ]


for i in model_lst: # 0,1,2,3
    # load model
    pre_model = i
    tokenizer = AutoTokenizer.from_pretrained(pre_model)
    model = AutoModelForSequenceClassification.from_pretrained(pre_model)
    sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

    print('====='+ '\033[95m' + pre_model + '\033[0m' + '=====')

    # target reviews
    review_list = list(tsh.text)

    tsh[pre_model] = '-'
    # predict
    for idx, review in tqdm(enumerate(review_list)):
        pred = sentiment_classifier(review)
        # print(f'{review}\n>> {pred[0]}')
        if (pred[0]['label']=='1')&(pred[0]['score'] > 0.7) | (pred[0]['label']=='LABEL_1')&(pred[0]['score'] > 0.7):
            tsh[pre_model][idx] = '1'
        elif (pred[0]['label']=='0')&(pred[0]['score']>0.7) | (pred[0]['label']=='LABEL_0')&(pred[0]['score'] > 0.7):  
            tsh[pre_model][idx] = '-1'
        else:
            tsh[pre_model][idx] = '0'

        # print(f'{review}\n>> {pred[0]}')

print('🥕')
print('')


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


=====jaehyeong/koelectra-base-v3-generalized-sentiment-analysis=====


0it [00:00, ?it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '1'
6it [00:00, 13.22it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '-1'
68it [00:03, 16.04it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

=====klue/bert-base=====


0it [00:00, ?it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '0'
2600it [02:29, 17.39it/s]


...

=====skt/kogpt2-base-v2=====


0it [00:00, ?it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '0'
6it [00:00, 11.57it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '-1'
2600it [02:48, 15.46it/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions an

=====beomi/kcbert-large=====


0it [00:00, ?it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '0'
53it [00:06, 11.29it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_6608\1483638988.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsh[pre_model][idx] = '-1'
2600it [06:53,  6.28it/s]

🥕



In [7]:
for i in tsh.iloc[:, :].columns:
    print(tsh[[i]].value_counts())
    print()

text                                                                                                                                                                                                                                            
정말 좋네요! 정말 좋네요!                                                                                                                                                                                                                                     7
최고에요! 최고에요!                                                                                                                                                                                                                                         6
좋아요~ 좋아요~                                                                                                                                                                                                                                           5
좋아용 연수 추천합니다 좋아용 연수 추

In [9]:
tsh.iloc[:,1:]

,score,jaehyeong/koelectra-base-v3-generalized-sentiment-analysis,klue/bert-base,matthewburke/korean_sentiment,monologg/koelectra-base-v2-discriminator,skt/kogpt2-base-v2,beomi/kcbert-large
0,1,1,0,1,0,0,0
1,1,1,0,1,0,0,0
2,1,1,0,1,0,0,0
3,1,1,0,1,0,0,0
4,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...
2595,1,1,0,1,0,0,-1
2596,1,1,0,1,0,0,0
2597,1,0,0,1,0,0,0
2598,1,1,0,1,0,0,0


In [10]:
tsh.to_csv('tsh_model.csv')